Basato su EVALITA 2020 HaSpeeDe (http://www.di.unito.it/~tutreeb/haspeede-evalita20/index.html)

Sviluppare un classificatore basato su SVM lineari che prende in input una rappresentazione
del testo costruita attraverso l’uso dei word embedding
(http://www.italianlp.it/resources/italian-word-embeddings/). Riportare i seguenti risultati:
- Testare diverse rappresentazioni del testo che variano rispetto al modo di combinare gli
embedding delle singole parole e/o rispetto alle categorie grammaticali delle parole
prese in considerazione. Valutare i diversi sistemi con un processo di 5-fold cross
validation condotto sul training set;
- Valutazione sul test set ufficiale del miglior sistema rispetto ai risultati ottenuti con il
processo di 5-fold cross validation del punto sopra.

I word embeddings scelti sono stati gli Italian Twitter Embeddings raccolti dal ItaliaNLP Lab (http://www.italianlp.it/download-italian-twitter-embeddings/).

In [1]:
#importo sqlite e specifico i path per accedere ai word embeddings e per salvarli come .txt
import sqlite3

sql_path = "word_embeddings/twitter128.sqlite"
txt_path = "word_embeddings/twitter128.txt"

In [2]:
#connessione al database sql "twitter128"
con = sqlite3.connect(sql_path)
#cursore per la connessione al database
cur = con.cursor()

In [3]:
#scrivo i dati dal database su un file di testo
with open(txt_path, 'w+') as out_file:
    for embedding in cur.execute("SELECT * FROM store"):
        str_embedding = [str(el) for el in embedding[:-1]]
        out_file.write('\t'.join(str_embedding)+'\n')

out_file.close()

Il file di testo twitter128.txt ha una riga per parola, in cui il primo elemento è la parola e i successivi 128 sono le componenti dell'embedding.

In [4]:
import numpy as np
import os

In [5]:
#creo una funzione per caricare i word embeddings a partire dal file di testo
def load_word_embeddings(src_path):
    embeddings = dict()
    #itero sulle righe del file
    for line in open(src_path, 'r'):
        #rimuovo spazi bianchi e separo la stringa in base ai tab, creando una lista in cui l'elemento 0 è la parola e gli elementi da 1 in poi sono le componenti dell'embedding
        line = line.strip().split('\t')
        word = line[0]
        embedding = line[1:]
        embedding = [float(comp) for comp in embedding] #convertiamo le componenti dell'embedding in float
        embeddings[word] = np.asarray(embedding) #trasformiamo la lista delle componenti in un vettore di numpy
    return embeddings
#embeddings è un dizionario che ha la parola come chiave e le componenti dell'embedding per quella parola in un array numpy come valore

In [6]:
#carico gli embeddings
embeddings = load_word_embeddings(txt_path)
#stampo gli embeddings della parola "ciao" per vedere se tutto è avvenuto correttamente
embeddings["ciao"]

array([ 1.00588687e-02,  9.70248729e-02,  2.28946786e-02, -1.01958990e-01,
        1.01152070e-01, -3.84831540e-02, -1.31021678e-01,  6.18477315e-02,
       -2.02476829e-02,  5.94022200e-02, -1.06382497e-01,  9.90854874e-02,
       -4.53294665e-02, -8.76475945e-02,  2.27265339e-02, -3.18616405e-02,
       -1.52194872e-01,  6.49362504e-02,  9.82077718e-02, -1.14106350e-01,
        1.01975193e-02, -1.52585441e-02,  1.42587781e-01,  5.78809668e-05,
        7.77927646e-03,  1.13248117e-01, -2.31339168e-02, -7.06618577e-02,
        1.82080530e-02,  6.44217804e-02, -1.78829823e-02, -3.65133770e-02,
        1.48065194e-01, -1.08065717e-01,  7.60474950e-02,  1.62513033e-01,
        1.47792243e-03, -8.97888169e-02,  1.41946062e-01, -3.19051333e-02,
        2.27965806e-02, -3.13461348e-02,  5.34125715e-02,  3.67401130e-02,
       -1.44961536e-01, -7.24047348e-02, -2.31131930e-02, -3.72798480e-02,
       -8.22194964e-02,  3.99212092e-02,  1.12063788e-01,  1.43978551e-01,
        9.28448662e-02,  

In [7]:
#carico il mio dataset dalla cartella con i file .conllu utilizzata anche per la scorsa task
conllu_dir = 'profiling_output/11925'
#inizializzo una lista che andrò a riempire coi percorsi dei file .conllu
all_documents_paths = []
#ottengo, scorrendo la directory, una lista completa dei percorsi per accedere ai vari file
for file_name in os.listdir(conllu_dir):
    file_path = os.path.join(conllu_dir, file_name)
    all_documents_paths.append(file_path)

In [8]:
import re

#funzione per la normalizzazione di numeri. la scrivo per richiamarla dentro normalize_text
def get_digits(text):
    try:
        #prova a convertire il testo in intero e lo salva eventualmente in una variabile val
        val = int(text)
    except:
        #se la conversione fallisce, sostituisce tutte le cifre nel testo con la stringa @Dg (\d corrisponde a una singola cifra)
        text = re.sub('\d', '@Dg', text)
        return text
    #se il valore è compreso tra 0 e 9999, lo restituisce come stringa. altrimenti, restituisce una stringa del tipo DIGLEN_Lunghezza, in base alla lunghezza della stringa calcolata con len
    if val >= 0 and val < 9999:
        return str(val)
    else:
        return "DIGLEN_" + str(len(str(val)))

#funzione per la normalizzazione del testo
def normalize_text(word):
    #controlla se la stringa è un url, se è presente al suo interno http oppure sia . che /. se lo è, la sostituisce con "___URL___" e la ritorna.
    if "http" in word or ("." in word and "/" in word):
        word = str("___URL___")
        return word
    #controlla se la stringa ha più di 26 caratteri. se sì, la sostituisce con "__LONG-LONG__" e la ritorna.
    if len(word) > 26:
        return "__LONG-LONG__"
    #richiama get_digits e memorizzo il risultato nella variabile new_word
    new_word = get_digits(word)
    #controlla se new_word è diversa da word e quindi se get_digits l'ha cambiata. se sì, aggiorno word (get_digits la cambia se la parola contiene sia cifre numeriche che altri caratteri, o se è un numero superiore a 9999).
    if new_word != word:
        word = new_word
    #se la prima lettera è maiuscola, capitalizza la parola (prima maiuscola, il resto minuscole)
    if word[0].isupper():
        word = word.capitalize()
    #se la prima lettera non è maiuscola (quindi è minuscola), rende tutta la parola in lettere minuscole
    else:
        word = word.lower()
    return word

In [9]:
def get_tokens_from_file(src_path):
    #inizializzo la lista dei token, per ora vuota
    document_tokens = []
    #lines_to_skip serve per tenere traccia delle righe da saltare
    lines_to_skip = 0
    #take_pos serve per tenere traccia se prendere o meno la POS della prossima parola
    take_pos = False
    #itero sulle righe del file
    for line in open(src_path, 'r'):
        #controllo se il primo carattere è una cifra (dovrebbe esserlo)
        if line[0].isdigit():
            #rimuove gli spazi bianchi all'inizio e alla fine della riga e la trasforma in una lista, con gli elementi separati da tab
            splitted_line = line.strip().split('\t')
            if '-' in splitted_line[0]:
                #se ho trovato un -, trasformo il primo elemento della lista a sua volta in una lista, separata da -
                skip_ids = splitted_line[0].split('-')
                #aggiorno lines_to_skip per sapere quante righe saltare
                lines_to_skip = int(skip_ids[1]) - int(skip_ids[0]) + 1 #l'indice ci indica quali righe saltare
                take_pos = True #booleano che indica che dobbiamo prendere la pos della prossima parola
                #normalizzo la parola riprendendo la funzione normalize_text
                word = normalize_text(splitted_line[1])
                #estraggo la pos della parola
                pos = splitted_line[3]
                #creo il token come dizionario, alla chiave "word" metto la parola, alla chiave "pos" metto un placeholder
                token = {
                    'word': word,
                    'pos': '_'
                }
                #aggiungo il token alla lista dei token
                document_tokens.append(token)
            else: #quindi no trattino nel primo elemento
                if lines_to_skip == 0: #se non ci sono righe da saltare
                    #normalizzo la parola riprendendo la funzione normalize_text
                    word = normalize_text(splitted_line[1])
                    #estraggo la pos della parola
                    pos = splitted_line[3]
                    #creo il token come dizionario, alla chiave "word" metto la parola, alla chiave "pos" metto il pos
                    token = {
                        'word': word,
                        'pos': pos
                    }
                    #aggiungo il token alla lista dei token
                    document_tokens.append(token)
                if take_pos:
                    pos = splitted_line[3]
                    #aggiorno il pos del token precedente
                    document_tokens[-1]['pos'] = pos
                    #reimposto take_pos come falso
                    take_pos = False
                #decrementa di 1 lines_to_skip, ma, se diventa negativo, lo riporta a 0
                lines_to_skip = max(0, lines_to_skip-1)
    return document_tokens

#devo fare così per evitare di avere confusione nel caso di parole composte, che non hanno POS e i cui numeri hanno un trattino, perché sono seguite dalle singole componenti; per fare un esempio:
# 5-6	farla	_	_	_	_	_	_	_	_
# 5	far	fare	VERB	V	VerbForm=Inf	2	advcl	_	_
# 6	la	lo	PRON	PC	Clitic=Yes|Gender=Fem|Number=Sing|Person=3|PronType=Prs	5	obj	_	_

In [10]:
#lista che conterrà i tokens di ogni documento, per ora vuota
all_documents = []

#estraggo tutti i tokens da ogni documento di all_documents_path e li appendo nella lista all_documents
for document_path in all_documents_paths:
    document_tokens = get_tokens_from_file(document_path)
    all_documents.append(document_tokens)
    
#stampo tutti i tokens del documento alla posizione 111 per verificare che tutto sia stato eseguito correttamente
all_documents[111]

[{'word': '.', 'pos': 'PUNCT'},
 {'word': '#pianominniti', 'pos': 'PROPN'},
 {'word': 'prevede', 'pos': 'VERB'},
 {'word': 'regalia', 'pos': 'NOUN'},
 {'word': 'di', 'pos': 'ADP'},
 {'word': '@dg@dg@dg@dg€', 'pos': 'PROPN'},
 {'word': 'a', 'pos': 'ADP'},
 {'word': '#immigrati', 'pos': 'NOUN'},
 {'word': 'X', 'pos': 'PROPN'},
 {'word': 'rimpatrio', 'pos': 'NOUN'},
 {'word': 'volontario+@dg@dg@dg€', 'pos': 'PROPN'},
 {'word': 'X', 'pos': 'ADJ'},
 {'word': 'rifarsi', 'pos': 'VERB'},
 {'word': '1', 'pos': 'NUM'},
 {'word': 'vita', 'pos': 'NOUN'},
 {'word': 'in', 'pos': 'ADP'},
 {'word': 'patria.viaggio', 'pos': 'NOUN'},
 {'word': 'sempre', 'pos': 'ADV'},
 {'word': 'a', 'pos': 'ADP'},
 {'word': 'spese', 'pos': 'NOUN'},
 {'word': 'nostre', 'pos': 'DET'},
 {'word': '.', 'pos': 'PUNCT'},
 {'word': '😡', 'pos': 'SYM'}]

In [11]:
#calcola la media degli embeddings di un documento
def compute_embeddings_mean(document_embeddings):
    #sum_array è la somma degli embeddings
    sum_array = np.sum(document_embeddings, axis=0)
    #la media è la somma degli embeddings divisa per il numero di embeddings
    mean_array = np.divide(sum_array, len(document_embeddings))
    return mean_array

In [12]:
#1° metodo di calcolo media: calcola la media di tutti i word embeddings

def compute_all_embeddings_mean(document_tokens):
    #inizializzo una lista che conterrà gli embeddings dei tokens del documento
    document_embeddings = []
    
    #itero sui tokens
    for token in document_tokens:
        #estraggo la parola dal dizionario token
        word = token['word']
        #controllo se la parola è presente nei miei embeddings
        if word in embeddings:
            #se la parola è presente nei miei embeddings, la appendo alla lista document_embeddings
            document_embeddings.append(embeddings[word])
    
    #controllo se document_embeddings è vuoto (quindi nessuna parola del documento ha un embeddings associato nei miei embeddings)
    if len(document_embeddings) == 0:
        #se document_embeddings è vuoto, creo un array numpy di 0 (di dimensione 128)
        mean_document_embeddings = np.zeros(128)
    else:
        #se document_embeddings non è vuoto, chiamo la funzione compute_document_mean per calcolare la media degli embeddings
        mean_document_embeddings = compute_embeddings_mean(document_embeddings)
    return mean_document_embeddings

In [13]:
#2° metodo di calcolo media: calcola la media degli embeddings solo di aggettivi, nomi e verbi

def compute_filtered_embeddings_mean(document_tokens):
    #inizializzo una lista che conterrà gli embeddings dei tokens del documento
    document_embeddings = []
    
    #itero sui tokens
    for token in document_tokens:
        #estraggo sia la parola che la POS dal dizionario token
        word = token['word']
        pos = token['pos']
        #controllo se la parola è presente nei miei embeddings e se il POS è uno tra aggettivo, nome o verbo
        if word in embeddings and pos in ['ADJ', 'NOUN', 'VERB']:
            #se le condizioni sopra sono vere, aggiungo la parola alla lista document_embeddings
            document_embeddings.append(embeddings[word])
    
    #controllo se document_embeddings è vuoto (quindi nessuna parola del documento ha un embeddings associato nei miei embeddings)
    if len(document_embeddings) == 0:
        #se document_embeddings è vuoto, creo un array numpy di 0 (di dimensione 128)
        mean_document_embeddings = np.zeros(128)
    else:
        #se document_embeddings non è vuoto, chiamo la funzione compute_document_mean per calcolare la media degli embeddings
        mean_document_embeddings = compute_embeddings_mean(document_embeddings)
    return mean_document_embeddings

In [14]:
#3° metodo di calcolo media: calcola la media degli embeddings di aggettivi, nomi e verbi separatamente, e concatena i 3 vettori ottenuti

def compute_filtered_embeddings_sep_means(document_tokens):
    #inizializzo tre diverse liste per contenere embeddings, una per aggettivi, una per nomi, e una per verbi
    adj_embeddings = []
    noun_embeddings = []
    verb_embeddings = []
    
    #itero sui tokens
    for token in document_tokens:
        #estraggo sia la parola che la POS dal dizionario token
        word = token['word']
        pos = token['pos']
        #controllo se la parola è presente nei miei embeddings e se il POS è aggettivo
        if word in embeddings and pos in ['ADJ']:
            #se le condizioni sopra sono vere, aggiungo la parola alla lista adj_embeddings
            adj_embeddings.append(embeddings[word])
        #controllo se la parola è presente nei miei embeddings e se il POS è nome
        elif word in embeddings and pos in ['NOUN']:
            #se le condizioni sopra sono vere, aggiungo la parola alla lista noun_embeddings
            noun_embeddings.append(embeddings[word])
        #controllo se la parola è presente nei miei embeddings e se il POS è verbo
        elif word in embeddings and pos in ['VERB']:
            #se le condizioni sopra sono vere, aggiungo la parola alla lista verb_embeddings
            verb_embeddings.append(embeddings[word])
    
    #per ognuna delle tre liste contenenti embeddings, controllo se sono vuote; se sì, creo un array numpy di 0 (di dimensione 128), altrimenti chiamo la funzioen compute_document_mean per calcolare la media degli embeddings
    if len(adj_embeddings) == 0:
        mean_adj_embeddings = np.zeros(128)
    else:
        mean_adj_embeddings = compute_embeddings_mean(adj_embeddings)
        
    if len(noun_embeddings) == 0:
        mean_noun_embeddings = np.zeros(128)
    else:
        mean_noun_embeddings = compute_embeddings_mean(noun_embeddings)
        
    if len(verb_embeddings) == 0:
        mean_verb_embeddings = np.zeros(128)
    else:
        mean_verb_embeddings = compute_embeddings_mean(verb_embeddings)  
    
    #eseguo la concatenazione degli embeddings medi di aggettivi, nomi e verbi. saranno quindi presenti 128x3=384 valori.
    mean_document_embeddings = np.concatenate([mean_adj_embeddings, mean_noun_embeddings, mean_verb_embeddings], axis=None)
    return mean_document_embeddings

In [15]:
#stampo delle prove calcolando la media degli embeddings dei token del primo documento nelle varie modalità per vedere che tutto abbia funzionato correttamente
doc_0_tokens = all_documents[0]
print('Numero e media di tutti gli embeddings:')
print(len(compute_all_embeddings_mean(doc_0_tokens)))
print(compute_all_embeddings_mean(doc_0_tokens))
print('\nNumero e media embeddings parole piene:')
print(len(compute_filtered_embeddings_mean(doc_0_tokens)))
print(compute_filtered_embeddings_mean(doc_0_tokens))
print('\nNumero e media separate embeddings parole piene:')
print(len(compute_filtered_embeddings_sep_means(doc_0_tokens)))
print(compute_filtered_embeddings_sep_means(doc_0_tokens))

Numero e media di tutti gli embeddings:
128
[-0.02775776  0.03938989 -0.00629901 -0.09540768 -0.02298989 -0.04438516
 -0.02631047 -0.02981493  0.06303736 -0.06666578 -0.02740279  0.05675423
  0.01849988 -0.07019756  0.00514951 -0.00465845  0.04153982 -0.0056832
 -0.00995647  0.0284641  -0.04878962 -0.05391109 -0.02918607  0.010651
  0.08411159 -0.00133598 -0.04336065 -0.02887877 -0.04764055 -0.02627937
  0.06848478  0.08748336  0.00389155 -0.0065819   0.06068013  0.01936245
  0.02808932 -0.01449003 -0.01192433 -0.06747614  0.04377068  0.0305988
 -0.09687228 -0.00313677 -0.05054593 -0.01249226 -0.00455744 -0.0032782
  0.09162627 -0.05142696  0.02338257  0.09695108  0.0091481   0.01609148
  0.05263629  0.06450496  0.05652132  0.04860144  0.01394459 -0.009985
 -0.03681103 -0.0579086   0.08197123  0.06187929  0.00200779 -0.07084321
 -0.0002449  -0.01253242 -0.01165658 -0.01927183  0.00799208  0.0491647
 -0.00693922 -0.02615696  0.03332344  0.06120816  0.01451376 -0.03288106
 -0.05986465 -0

In [16]:
#funzione per estrarre features dai documenti. utilizza una delle 3 funzioni precedenti per ottenere la media degli embedding a seconda del parametro method, che può essere "all", "filtered" o "filtered_sep"
def extract_features(documents, method):
    dataset_features = []
    for document_tokens in documents:
        if method == "all":
            document_embeddings = compute_all_embeddings_mean(document_tokens)
        elif method == "filtered":
            document_embeddings = compute_filtered_embeddings_mean(document_tokens)
        elif method == "filtered_sep":
            document_embeddings = compute_filtered_embeddings_sep_means(document_tokens)
        else:
            raise Exception(f'Invalid element {method}')
        dataset_features.append(document_embeddings)
    return dataset_features

In [17]:
#tentativo 1, calcola la media di tutti i word embeddings.
all_features = extract_features(all_documents, "all")

#stampo il numero dei documenti e il numero di features all'interno di uno per verificare che tutto sia andato correttamente
len(all_features), len(all_features[0])

(6837, 128)

In [18]:
#creo una lista di etichette in base ai nomi dei documenti
def create_label_list(all_documents_paths):
    labels = []
    for document_path in all_documents_paths:
        document_path = document_path[:-len('.conllu')]
        #divido ogni file path, senza .conllu, in base al #, trasformandolo in una lista che ha l'id a posizione 0 e hs a posizione 1
        splitted_file_path = document_path.split('#')
        #salvo hs nella variabile hs
        hs = splitted_file_path[1]
        #appendo il valore di hs nella lista delle labels
        labels.append(hs)
    return labels

In [19]:
#creo la lista delle labels e ne stampo 15 per vedere se tutto è andato a buon fine
all_labels = create_label_list(all_documents_paths)
all_labels[:15]

['0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1']

In [20]:
#normalizzo le features di train attraverso il MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(all_features)

#stampo le dimensioni dell'array X_train per verificare che tutto sia andato correttamente
X_train.shape

(6837, 128)

In [21]:
#importo le librerie per fare k-fold cross validation
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(X_train))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [X_train[idx] for idx in train_ids]
    fold_y_train = np.asarray([all_labels[idx] for idx in train_ids])
    fold_X_test = [X_train[idx] for idx in test_ids]
    fold_y_test = np.asarray([all_labels[idx] for idx in test_ids])
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.77046783625731, baseline: 0.5986842105263158
Accuracy fold 2: 0.7470760233918129, baseline: 0.591374269005848
Accuracy fold 3: 0.7534747622531089, baseline: 0.60424286759327
Accuracy fold 4: 0.7556693489392831, baseline: 0.5874177029992684
Accuracy fold 5: 0.7681053401609363, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      4071
           1       0.71      0.68      0.70      2766

    accuracy                           0.76      6837
   macro avg       0.75      0.75      0.75      6837
weighted avg       0.76      0.76      0.76      6837



In [22]:
#tentativo 2, calcola la media dei word embeddings solo di aggettivi, nomi e verbi.
all_features = extract_features(all_documents, "filtered")

#stampo il numero dei documenti e il numero di features all'interno di uno per verificare che tutto sia andato correttamente
len(all_features), len(all_features[0])

(6837, 128)

In [23]:
#normalizzo le features di train attraverso il MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(all_features)

#stampo le dimensioni dell'array X_train per verificare che tutto sia andato correttamente
X_train.shape

(6837, 128)

In [24]:
#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(X_train))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [X_train[idx] for idx in train_ids]
    fold_y_train = np.asarray([all_labels[idx] for idx in train_ids])
    fold_X_test = [X_train[idx] for idx in test_ids]
    fold_y_test = np.asarray([all_labels[idx] for idx in test_ids])
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.7368421052631579, baseline: 0.5986842105263158
Accuracy fold 2: 0.7412280701754386, baseline: 0.591374269005848
Accuracy fold 3: 0.7271397220190198, baseline: 0.60424286759327
Accuracy fold 4: 0.7278712509144111, baseline: 0.5874177029992684
Accuracy fold 5: 0.731528895391368, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.76      0.80      0.78      4071
           1       0.68      0.63      0.66      2766

    accuracy                           0.73      6837
   macro avg       0.72      0.72      0.72      6837
weighted avg       0.73      0.73      0.73      6837



In [25]:
#tentativo 3, calcola la media separatamente dei word embedding di aggettivi, nomi e verbi, e concatena i 3 vettori ottenuti.
all_features = extract_features(all_documents, "filtered_sep")

#stampo il numero dei documenti e il numero di features all'interno di uno per verificare che tutto sia andato correttamente
len(all_features), len(all_features[0])

(6837, 384)

In [26]:
#normalizzo le features di train attraverso il MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(all_features)

#stampo le dimensioni dell'array X_train per verificare che tutto sia andato correttamente
X_train.shape

(6837, 384)

In [27]:
#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(X_train))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [X_train[idx] for idx in train_ids]
    fold_y_train = np.asarray([all_labels[idx] for idx in train_ids])
    fold_X_test = [X_train[idx] for idx in test_ids]
    fold_y_test = np.asarray([all_labels[idx] for idx in test_ids])
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.7258771929824561, baseline: 0.5986842105263158
Accuracy fold 2: 0.7397660818713451, baseline: 0.591374269005848
Accuracy fold 3: 0.7161667885881492, baseline: 0.60424286759327
Accuracy fold 4: 0.7022677395757132, baseline: 0.5874177029992684
Accuracy fold 5: 0.7205559619604974, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      4071
           1       0.67      0.62      0.64      2766

    accuracy                           0.72      6837
   macro avg       0.71      0.70      0.71      6837
weighted avg       0.72      0.72      0.72      6837



Dei 3 tentativi, quello usando la media di tutti i word embeddings ha portato i migliori risultati in termini di prestazioni. Lo riportiamo dunque sui test set ufficiali del task.

In [28]:
#carico i miei test set dalla cartella con i file .conllu
conllu_dir_test_1 = 'profiling_output_test_1/11936'
conllu_dir_test_2 = 'profiling_output_test_2/11937'
#inizializzo due liste che andrò a riempire coi percorsi dei file .conllu
all_documents_paths_test_1 = []
all_documents_paths_test_2 = []
#ottengo, per le due directory, una lista completa dei percorsi per accedere ai vari file
for file_name in os.listdir(conllu_dir_test_1):
    file_path = os.path.join(conllu_dir_test_1, file_name)
    all_documents_paths_test_1.append(file_path)
for file_name in os.listdir(conllu_dir_test_2):
    file_path = os.path.join(conllu_dir_test_2, file_name)
    all_documents_paths_test_2.append(file_path)

In [29]:
#due liste, una per ogni test set, che conterranno i tokens di ogni documento, per ora vuote
all_documents_test_1 = []
all_documents_test_2 = []

#estraggo tutti i tokens da ogni documento di all_documents_path_test_1 e li appendo nella lista all_documents_test_1
for document_path in all_documents_paths_test_1:
    document_tokens = get_tokens_from_file(document_path)
    all_documents_test_1.append(document_tokens)
    
#estraggo tutti i tokens da ogni documento di all_documents_path_test_2 e li appendo nella lista all_documents_test_2
for document_path in all_documents_paths_test_2:
    document_tokens = get_tokens_from_file(document_path)
    all_documents_test_2.append(document_tokens)
    
#stampo tutti i tokens dei due documenti alla posizione 50 per verificare che tutto sia stato eseguito correttamente
all_documents_test_1[50], all_documents_test_2[50]

([{'word': 'Il', 'pos': 'DET'},
  {'word': 'manifesto', 'pos': 'NOUN'},
  {'word': 'per', 'pos': 'ADP'},
  {'word': 'gli', 'pos': 'DET'},
  {'word': 'immigrati', 'pos': 'NOUN'},
  {'word': ':', 'pos': 'PUNCT'},
  {'word': '\u200b"', 'pos': 'PUNCT'},
  {'word': 'Se', 'pos': 'SCONJ'},
  {'word': 'sei', 'pos': 'AUX'},
  {'word': 'venuto', 'pos': 'VERB'},
  {'word': 'per', 'pos': 'ADP'},
  {'word': 'delinquere', 'pos': 'VERB'},
  {'word': 'faremo', 'pos': 'VERB'},
  {'word': 'di', 'pos': 'ADP'},
  {'word': 'tutto', 'pos': 'PRON'},
  {'word': 'per', 'pos': 'ADP'},
  {'word': 'cacciarti', 'pos': 'VERB'},
  {'word': '"', 'pos': 'PUNCT'}],
 [{'word': 'Mentre', 'pos': 'SCONJ'},
  {'word': 'sto', 'pos': 'VERB'},
  {'word': 'per', 'pos': 'ADP'},
  {'word': 'mandarla', 'pos': 'NOUN'},
  {'word': 'a', 'pos': 'ADP'},
  {'word': 'fanculo', 'pos': 'NOUN'},
  {'word': 'fa', 'pos': 'ADV'},
  {'word': ':', 'pos': 'PUNCT'},
  {'word': 'eh', 'pos': 'INTJ'},
  {'word': 'pensi', 'pos': 'VERB'},
  {'word': 'c

In [30]:
#addestro un svc lineare sul mio sistema migliore in modo da poterlo testare sui test set ufficiali

all_features = extract_features(all_documents, "all")
#stampo il numero dei documenti e il numero di features all'interno di uno per verificare che tutto sia andato correttamente
len(all_features), len(all_features[0])

(6837, 128)

In [31]:
#creo la lista delle labels e ne stampo 15 per vedere se tutto è andato a buon fine
all_labels = create_label_list(all_documents_paths)
all_labels[:15]

['0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1']

In [32]:
#normalizzo tutte le features attraverso il MinMaxScaler
scaler = MinMaxScaler()
all_features = scaler.fit_transform(all_features)

#stampo le dimensioni dell'array all_features.shape per verificare che tutto sia andato correttamente
all_features.shape

(6837, 128)

In [33]:
#modello svc lineare
svc_model = LinearSVC(dual=False)
#addestro il modello sui dati di train
svc_model.fit(all_features, all_labels)

LinearSVC(dual=False)

In [34]:
#estraggo le features dai documenti di test
all_features_test_1 = extract_features(all_documents_test_1, "all")
all_features_test_2 = extract_features(all_documents_test_2, "all")

#normalizzo le features con il MinMaxScaler già fittato sui dati di train
all_features_test_1 = scaler.transform(all_features_test_1)
all_features_test_2 = scaler.transform(all_features_test_2)

#stampo le dimensioni degli array per verificare che tutto sia andato correttamente
all_features_test_1.shape, all_features_test_2.shape

((500, 128), (1263, 128))

In [35]:
#creo la lista delle labels per i due test sets
all_labels_test_1 = create_label_list(all_documents_paths_test_1)
all_labels_test_2 = create_label_list(all_documents_paths_test_2)

#stampo 15 labels per ogni test set in modo da garantire che tutto sia andato correttamente
all_labels_test_1[:15], all_labels_test_2[:15]

(['0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0'],
 ['1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0'])

In [36]:
#predizione sul test1
y_pred_test_1 = svc_model.predict(all_features_test_1) #predicto sull'svc_model già fittato ai dati di train
print(classification_report(all_labels_test_1, y_pred_test_1))

              precision    recall  f1-score   support

           0       0.73      0.95      0.82       319
           1       0.80      0.37      0.51       181

    accuracy                           0.74       500
   macro avg       0.76      0.66      0.66       500
weighted avg       0.75      0.74      0.71       500



In [37]:
#predizione sul test2
y_pred_test_2 = svc_model.predict(all_features_test_2) #predicto sull'svc_model già fittato ai dati di train
print(classification_report(all_labels_test_2, y_pred_test_2))

              precision    recall  f1-score   support

           0       0.73      0.76      0.74       641
           1       0.74      0.72      0.73       622

    accuracy                           0.74      1263
   macro avg       0.74      0.74      0.74      1263
weighted avg       0.74      0.74      0.74      1263

